<a href="https://colab.research.google.com/github/shreyasrajesh0308/ece209as_project/blob/main/software/LSTM_UCIHAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LSTM 

Building the LSTM architecture and experimenting with the UCI HAR dataset

Results are compared for the following cases:

1. Baseline LSTM model.
2. Model with weight decay
3. Model with Rotation. 
4. Model with Scale. 
5. Model with Magnify 
6. Model with Resample
7. Model with Noise

|  Activity |  Baseline |  Weight Decay |  Rotation | Scaling  |  Noise |  Resampling | Magnify | 
|---|---|---|---|---|---|---|---|
| Walking  | 0.94  |  0.95 |  0.96 |  0.96 |  0.95 |  0.94 | 0.97 | 
| Stairs-Up  |  0.96 |  0.97 | 0.97 | 0.98  |  0.95 | 0.96  | 0.98 |
| Stairs-Down  | .99  | 0.99  |  0.99 |  0.99 |  0.99 | 1.0  | 0.99 |
| Sitting  | 0.80  | 0.77  |  0.8 | 0.82  | 0.79  |  0.79 | 0.81 |
| Standing | 0.83  |  0.86 |  0.79 |  0.9 | 0.81  |  0.83 | 0.94 |
| Lying  |  1 |  0.99 |  0.94 | 1  | 1  |  1 | 1 |
|  Overall Accuracy |  0.92 |  0.92 | 0.9  | 0.94  |  0.91 | 0.92  | 0.95|

## Module Import

In [ ]:
import tensorflow as tf
from tensorflow import keras 
from keras import layers

import numpy as np

from sklearn.utils import class_weight

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the Dataset

In [ ]:
path_to_dataset = "/content/drive/My Drive/ECE_209_AS_project_data/UCIHAR/"
path_to_logs = "/content/drive/My Drive/ECE_209_AS_project_data/UCIHARlogs/"
path_to_models = "/content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/"

# Load only accelero and Gyro Data, we dont use Magno
X_train = np.load(path_to_dataset + "X_train.npy")[:,:,:9]
# First argmax
Y_train = np.load(path_to_dataset + "y_train.npy").argmax(1)

X_test = np.load(path_to_dataset + "X_test.npy")[:,:,:9]
Y_test = np.load(path_to_dataset + "y_test.npy").argmax(1)

In [ ]:
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(7352, 128, 9)
(7352,)
(2947, 128, 9)
(2947,)


## Randomy Permute the dataset

In [ ]:
data_class_names = ["Walking", "Stairs-Up", "Stairs-Down", "Sitting", "Standing", "Lying"]
data_class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=range(len(data_class_names)),y=Y_train)
data_class_weights = dict(zip(range(len(data_class_weights)),data_class_weights.round(2)))

In [ ]:
data_class_weights

{0: 1.0, 1: 1.14, 2: 1.24, 3: 0.95, 4: 0.89, 5: 0.87}

### Define the model 

In [ ]:
inputs = keras.Input(shape = (128, 9))
features = layers.LSTM(100)(inputs)
features = layers.Dropout(0.5)(features)
features = layers.Dense(100, activation = "relu")(features)
outputs = layers.Dense(6, activation = "softmax")(features)

model = keras.Model(inputs = inputs, outputs = outputs)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 9)]          0         
                                                                 
 lstm (LSTM)                 (None, 100)               44000     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense (Dense)               (None, 100)               10100     
                                                                 
 dense_1 (Dense)             (None, 6)                 606       
                                                                 
Total params: 54,706
Trainable params: 54,706
Non-trainable params: 0
_________________________________________________________________


### Split the Data and Train the model 

In [ ]:
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)

In [ ]:
seeds = []

In [ ]:
from sklearn.metrics import confusion_matrix, recall_score

In [ ]:
test_accuracies = []
recall_scores = []

for i in range(5):

  rnd_seed = np.random.randint(1, 100)
  seeds.append(rnd_seed)
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  train_data = X_train[len(X_train)//10:]
  train_labels = Y_train[len(X_train)//10:]
  val_data = X_train[:len(X_train)//10]
  val_labels = Y_train[:len(X_train)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"LSTM_baseline_{i}"), tensorboard]
  history = model.fit(train_data, train_labels, epochs = 25, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"LSTM_baseline_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_baseline_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_baseline_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_baseline_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_baseline_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_baseline_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_baseline_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_baseline_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_baseline_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_baseline_4/assets


tf.Tensor(0.90770274, shape=(), dtype=float32)


In [ ]:
# Print Average Total Accuracy 
sum_ac = 0

for ac in test_accuracies:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {sum_ac/len(test_accuracies)}")

Mean accuracy is 0.9182219266891479


In [ ]:
mean_recall_scores  = sum(recall_scores)/len(recall_scores)

for i in range(len(mean_recall_scores)):

  print(f"Label {data_class_names[i]} has mean accuracy {mean_recall_scores[i]}")

Label Walking has mean accuracy 0.9370967741935484
Label Stairs-Up has mean accuracy 0.9562632696390658
Label Stairs-Down has mean accuracy 0.9942857142857143
Label Sitting has mean accuracy 0.8061099796334013
Label Standing has mean accuracy 0.8278195488721805
Label Lying has mean accuracy 1.0


## Same model with regularization

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
inputs = keras.Input(shape = (128, 9))
features = layers.LSTM(100, kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(inputs)
features = layers.Dropout(0.5)(features)
features = layers.Dense(100, activation = "relu", kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(features)
outputs = layers.Dense(6, activation = "softmax", kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4))(features)

model_with_reg = keras.Model(inputs = inputs, outputs = outputs)

In [ ]:
test_accuracies_with_reg = []
recall_scores_with_reg = []

for i in range(5):

  rnd_seed = seeds[i]
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  train_data = X_train[len(X_train)//10:]
  train_labels = Y_train[len(X_train)//10:]
  val_data = X_train[:len(X_train)//10]
  val_labels = Y_train[:len(X_train)//10]

  model_with_reg = keras.Model(inputs = inputs, outputs = outputs)
  model_with_reg.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"LSTM_with_reg_{i}"), tensorboard]
  history = model_with_reg.fit(train_data, train_labels, epochs = 25, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"LSTM_with_reg_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_with_reg.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_with_reg.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_with_reg_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_with_reg_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_with_reg_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_with_reg_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_with_reg_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_with_reg_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_with_reg_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_with_reg_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_with_reg_4/assets


tf.Tensor(0.9144893, shape=(), dtype=float32)


In [ ]:
sum_ac = 0

for ac in test_accuracies_with_reg:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {sum_ac/len(test_accuracies_with_reg)}")

Mean accuracy is 0.9193077683448792


In [ ]:
mean_recall_scores_with_reg  = sum(recall_scores_with_reg)/len(recall_scores_with_reg)

for i in range(len(mean_recall_scores_with_reg)):

  print(f"Label {data_class_names[i]} has mean accuracy {mean_recall_scores_with_reg[i]}")

Label Walking has mean accuracy 0.9443548387096774
Label Stairs-Up has mean accuracy 0.9749469214437367
Label Stairs-Down has mean accuracy 0.9914285714285714
Label Sitting has mean accuracy 0.769857433808554
Label Standing has mean accuracy 0.8563909774436089
Label Lying has mean accuracy 0.9899441340782122


## With Augmentations

1. Rotation
2. Scale
3. Magnify
4. Resample
5. Noise

### Augmentations

In [ ]:
import numpy as np
import tensorflow as tf
import random

def resampling_fast_random(x):
    M, N = random.choice([[1, 0], [2, 1], [3, 2]])
    time_steps = x.shape[1]
    raw_set = np.arange(x.shape[1])
    interp_steps = np.arange(0, raw_set[-1] + 1e-1, 1 / (M + 1))
    x_interp = interp1d(raw_set, x, axis=1)
    x_up = x_interp(interp_steps)

    length_inserted = x_up.shape[1]
    start = random.randint(0, length_inserted - time_steps * (N + 1))
    index_selected = np.arange(start, start + time_steps * (N + 1), N + 1)
    return x_up[:, index_selected, :]

def resampling(x,M,N):
    '''
    :param x: the data of a batch,shape=(batch_size,timesteps,features)
    :param M: the number of  new value under tow values
    :param N: the interval of resampling
    :return: x after resampling，shape=(batch_size,timesteps,features)
    '''
    assert M>N,'the value of M have to greater than N'

    timesetps = x.shape[1]

    for i in range(timesetps-1):
        x1 = x[:,i*(M+1),:]
        x2 = x[:,i*(M+1)+1,:]
        for j in range(M):
            v = np.add(1,np.subtract(x2,x1)*(j+1)/(M+1))
            x = np.insert(x,i*(M+1)+j+1,v,axis=1)

    length_inserted = x.shape[1]
    start = random.randint(0,length_inserted-timesetps*(N+1))
    index_selected = np.arange(start,start+timesetps*(N+1),N+1)
    return x[:,index_selected,:]
    return x

def noise(x):
    x = tf.add(x,tf.multiply(x,tf.cast(tf.random.uniform(shape = (x.shape[0],x.shape[1],x.shape[2]),minval=-0.1,maxval=0.1),tf.float32)))
    return x


def scaling(x):
    alpha = np.random.randint(7,10)/10
    # alpha = 0.9
    return tf.multiply(x,alpha)
#
def magnify(x):
    lam = np.random.randint(11,14)/10
    return tf.multiply(x,lam)


def inverting(x):
    return np.multiply(x,-1)
def reversing(x):
    return x[:,-1::-1,:]


def rotation(x):
    c = x.shape[2]//3
    x_new = rotation_transform_vectorized(x[:,:,0:3])
    for i in range(1,c):
        temp = rotation_transform_vectorized(x[:,:,i*3:(i+1)*3])
        x_new = np.concatenate((x_new,temp),axis=-1)
    return x_new
def rotation_transform_vectorized(X):
    """
    Applying a random 3D rotation
    """
    axes = np.random.uniform(low=-1, high=1, size=(X.shape[0], X.shape[2]))
    angles = np.random.uniform(low=-np.pi, high=np.pi, size=(X.shape[0]))
    matrices = axis_angle_to_rotation_matrix_3d_vectorized(axes, angles)

    return np.matmul(X, matrices)
def axis_angle_to_rotation_matrix_3d_vectorized(axes, angles):
    """
    Get the rotational matrix corresponding to a rotation of (angle) radian around the axes
    Reference: the Transforms3d package - transforms3d.axangles.axangle2mat
    Formula: http://en.wikipedia.org/wiki/Rotation_matrix#Axis_and_angle
    """
    axes = axes / np.linalg.norm(axes, ord=2, axis=1, keepdims=True)
    x = axes[:, 0]; y = axes[:, 1]; z = axes[:, 2]
    c = np.cos(angles)
    s = np.sin(angles)
    C = 1 - c

    xs = x*s;   ys = y*s;   zs = z*s
    xC = x*C;   yC = y*C;   zC = z*C
    xyC = x*yC; yzC = y*zC; zxC = z*xC

    m = np.array([
        [ x*xC+c,   xyC-zs,   zxC+ys ],
        [ xyC+zs,   y*yC+c,   yzC-xs ],
        [ zxC-ys,   yzC+xs,   z*zC+c ]])
    matrix_transposed = np.transpose(m, axes=(2,0,1))
    return matrix_transposed
## This example using cubic splice is not the best approach to generate random curves. 
## You can use other aprroaches, e.g., Gaussian process regression, Bezier curve, etc.
def GenerateRandomCurves(X, sigma=0.2, knot=4):
    xx = (np.ones((X.shape[1],1))*(np.arange(0,X.shape[0], (X.shape[0]-1)/(knot+1)))).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, X.shape[1]))
    x_range = np.arange(X.shape[0])
    cs_x = CubicSpline(xx[:,0], yy[:,0])
    cs_y = CubicSpline(xx[:,1], yy[:,1])
    cs_z = CubicSpline(xx[:,2], yy[:,2])
    return np.array([cs_x(x_range),cs_y(x_range),cs_z(x_range)]).transpose()

  
def DistortTimesteps(X, sigma=0.2):
    tt = GenerateRandomCurves(X, sigma) # Regard these samples aroun 1 as time intervals
    tt_cum = np.cumsum(tt, axis=0)        # Add intervals to make a cumulative graph
    # Make the last value to have X.shape[0]
    t_scale = [(X.shape[0]-1)/tt_cum[-1,0],(X.shape[0]-1)/tt_cum[-1,1],(X.shape[0]-1)/tt_cum[-1,2]]
    tt_cum[:,0] = tt_cum[:,0]*t_scale[0]
    tt_cum[:,1] = tt_cum[:,1]*t_scale[1]
    tt_cum[:,2] = tt_cum[:,2]*t_scale[2]
    return tt_cum

def DA_TimeWarp(X, sigma=0.2):
    tt_new = DistortTimesteps(X, sigma)
    X_new = np.zeros(X.shape)
    x_range = np.arange(X.shape[0])
    X_new[:,0] = np.interp(x_range, tt_new[:,0], X[:,0])
    X_new[:,1] = np.interp(x_range, tt_new[:,1], X[:,1])
    X_new[:,2] = np.interp(x_range, tt_new[:,2], X[:,2])
    return X_new


### Rotation

In [ ]:
test_accuracies_rot = []
recall_scores_rot = []

for i in range(5):

  rnd_seed = np.random.randint(1, 100)
  seeds.append(rnd_seed)
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_rot = rotation((X_train))
  Xtrain_tot=np.concatenate((X_train, X_train_rot))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"LSTM_withrot_{i}"), tensorboard]
  history = model.fit(train_data, train_labels, epochs = 25, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"LSTM_withrot_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_rot.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_rot.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withrot_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withrot_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withrot_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withrot_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withrot_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withrot_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withrot_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withrot_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withrot_4/assets


tf.Tensor(0.92229384, shape=(), dtype=float32)


In [ ]:
sum_ac = 0

for ac in test_accuracies_rot:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_rot), 2)}")

Mean accuracy is 0.9


In [ ]:
mean_recall_scores_with_rot  = sum(recall_scores_rot)/len(recall_scores_rot)

for i in range(len(mean_recall_scores_with_rot)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_rot[i], 2)}")

Label Walking has mean accuracy 0.96
Label Stairs-Up has mean accuracy 0.97
Label Stairs-Down has mean accuracy 0.99
Label Sitting has mean accuracy 0.8
Label Standing has mean accuracy 0.79
Label Lying has mean accuracy 0.94


### Scaling

In [ ]:
test_accuracies_scale = []
recall_scores_scale = []

for i in range(5):

  rnd_seed = np.random.randint(1, 100)
  seeds.append(rnd_seed)
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_scale = scaling((X_train))
  Xtrain_tot=np.concatenate((X_train, X_train_scale))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"LSTM_withscale_{i}"), tensorboard]
  history = model.fit(train_data, train_labels, epochs = 25, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"LSTM_withscale_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_scale.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_scale.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withscale_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withscale_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withscale_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withscale_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withscale_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withscale_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withscale_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withscale_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withscale_4/assets


tf.Tensor(0.9412962, shape=(), dtype=float32)


In [ ]:
sum_ac = 0

for ac in test_accuracies_scale:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_scale), 2)}")

Mean accuracy is 0.94


In [ ]:
mean_recall_scores_with_scale  = sum(recall_scores_scale)/len(recall_scores_scale)

for i in range(len(mean_recall_scores_with_scale)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_scale[i], 2)}")

Label Walking has mean accuracy 0.96
Label Stairs-Up has mean accuracy 0.98
Label Stairs-Down has mean accuracy 0.99
Label Sitting has mean accuracy 0.82
Label Standing has mean accuracy 0.9
Label Lying has mean accuracy 1.0


### Magnify

In [ ]:
test_accuracies_magnify = []
recall_scores_magnify = []

for i in range(5):

  rnd_seed = np.random.randint(1, 100)
  seeds.append(rnd_seed)
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_magnify = magnify((X_train))
  Xtrain_tot=np.concatenate((X_train, X_train_magnify))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"LSTM_withmag_{i}"), tensorboard]
  history = model.fit(train_data, train_labels, epochs = 25, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"LSTM_withmag_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_magnify.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_magnify.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withmag_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withmag_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withmag_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withmag_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withmag_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withmag_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withmag_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withmag_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withmag_4/assets


tf.Tensor(0.94876146, shape=(), dtype=float32)


In [ ]:
test_accuracies_magnify = []
recall_scores_magnify = []

In [ ]:
for i in range(5):
  
  model_saved = keras.models.load_model(path_to_models + f"LSTM_withmag_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_magnify.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_magnify.append(recall_score(Y_test, prediction, average=None))

tf.Tensor(0.9477435, shape=(), dtype=float32)
tf.Tensor(0.9453682, shape=(), dtype=float32)
tf.Tensor(0.9433322, shape=(), dtype=float32)
tf.Tensor(0.956566, shape=(), dtype=float32)
tf.Tensor(0.94876146, shape=(), dtype=float32)


In [ ]:
sum_ac = 0

for ac in test_accuracies_magnify:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_magnify), 2)}")

Mean accuracy is 0.95


In [ ]:
mean_recall_scores_with_magnify  = sum(recall_scores_magnify)/len(recall_scores_magnify)

for i in range(len(mean_recall_scores_with_magnify)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_magnify[i], 2)}")

Label Walking has mean accuracy 0.97
Label Stairs-Up has mean accuracy 0.98
Label Stairs-Down has mean accuracy 0.99
Label Sitting has mean accuracy 0.81
Label Standing has mean accuracy 0.94
Label Lying has mean accuracy 1.0


### Resampling

In [ ]:
from sklearn.metrics import recall_score

In [ ]:
test_accuracies_resampling = []
recall_scores_resampling = []

for i in range(5):

  rnd_seed = np.random.randint(1, 100)
  seeds.append(rnd_seed)
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_resample = resampling((X_train), 1, 0)
  Xtrain_tot=np.concatenate((X_train, X_train_resample))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"LSTM_withresampling_{i}"), tensorboard]
  history = model.fit(train_data, train_labels, epochs = 25, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"LSTM_withresampling_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_resampling.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_resampling.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withresampling_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withresampling_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withresampling_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withresampling_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withresampling_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withresampling_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withresampling_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withresampling_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withresampling_4/assets


tf.Tensor(0.9127927, shape=(), dtype=float32)


In [ ]:
sum_ac = 0

for ac in test_accuracies_resampling:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_resampling), 2)}")

Mean accuracy is 0.92


In [ ]:
mean_recall_scores_with_resampling  = sum(recall_scores_resampling)/len(recall_scores_resampling)

for i in range(len(mean_recall_scores_with_resampling)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_resampling[i], 2)}")

Label Walking has mean accuracy 0.94
Label Stairs-Up has mean accuracy 0.96
Label Stairs-Down has mean accuracy 1.0
Label Sitting has mean accuracy 0.79
Label Standing has mean accuracy 0.83
Label Lying has mean accuracy 1.0


### Noise 

In [ ]:
test_accuracies_noise = []
recall_scores_noise = []

for i in range(5):

  rnd_seed = np.random.randint(1, 100)
  seeds.append(rnd_seed)
  tf.random.set_seed(rnd_seed)
  indices = np.random.RandomState(seed=rnd_seed).permutation(len(X_train))
  X_train = X_train[indices]
  Y_train = Y_train[indices]

  X_train_noise = noise((X_train))
  Xtrain_tot=np.concatenate((X_train, X_train_noise))
  Y_train_tot = np.concatenate((Y_train, Y_train))

  train_data = Xtrain_tot[len(Xtrain_tot)//10:]
  train_labels = Y_train_tot[len(Xtrain_tot)//10:]
  val_data = Xtrain_tot[:len(Xtrain_tot)//10]
  val_labels = Y_train_tot[:len(Xtrain_tot)//10]

  model = keras.Model(inputs = inputs, outputs = outputs)
  model.compile(loss = "sparse_categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])
  tensorboard = keras.callbacks.TensorBoard(log_dir = path_to_logs)
  callbacks = [keras.callbacks.ModelCheckpoint(filepath=path_to_models + f"LSTM_withnoise_{i}"), tensorboard]
  history = model.fit(train_data, train_labels, epochs = 25, batch_size = 64, validation_data = (val_data, val_labels), callbacks = callbacks, class_weight = data_class_weights, verbose = False)

  model_saved = keras.models.load_model(path_to_models + f"LSTM_withnoise_{i}")
  test_accuracy = tf.keras.metrics.Accuracy()
  logits = model_saved(X_test, training=False)
  prediction = tf.argmax(logits, axis=1, output_type=tf.int32)
  test_accuracies_noise.append(test_accuracy(prediction, Y_test))
  print(test_accuracy(prediction, Y_test))
  recall_scores_noise.append(recall_score(Y_test, prediction, average=None))

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withnoise_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withnoise_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withnoise_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withnoise_0/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withnoise_0/assets


...

INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withnoise_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withnoise_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withnoise_4/assets


INFO:tensorflow:Assets written to: /content/drive/My Drive/ECE_209_AS_project_data/UCIHARmodels/LSTM_withnoise_4/assets


tf.Tensor(0.9117747, shape=(), dtype=float32)


In [ ]:
sum_ac = 0

for ac in test_accuracies_noise:

  sum_ac += ac.numpy()

print(f"Mean accuracy is {round(sum_ac/len(test_accuracies_noise), 2)}")

Mean accuracy is 0.91


In [ ]:
mean_recall_scores_with_noise  = sum(recall_scores_noise)/len(recall_scores_noise)

for i in range(len(mean_recall_scores_with_noise)):

  print(f"Label {data_class_names[i]} has mean accuracy {round(mean_recall_scores_with_noise[i], 2)}")

Label Walking has mean accuracy 0.95
Label Stairs-Up has mean accuracy 0.95
Label Stairs-Down has mean accuracy 0.99
Label Sitting has mean accuracy 0.79
Label Standing has mean accuracy 0.81
Label Lying has mean accuracy 1.0


## Results

|  Activity |  Baseline |  Weight Decay |  Rotation | Scaling  |  Noise |  Resampling | Magnify | 
|---|---|---|---|---|---|---|---|
| Walking  | 0.94  |  0.95 |  0.96 |  0.96 |  0.95 |  0.94 | 0.97 | 
| Stairs-Up  |  0.96 |  0.97 | 0.97 | 0.98  |  0.95 | 0.96  | 0.98 |
| Stairs-Down  | .99  | 0.99  |  0.99 |  0.99 |  0.99 | 1.0  | 0.99 |
| Sitting  | 0.80  | 0.77  |  0.8 | 0.82  | 0.79  |  0.79 | 0.81 |
| Standing | 0.83  |  0.86 |  0.79 |  0.9 | 0.81  |  0.83 | 0.94 |
| Lying  |  1 |  0.99 |  0.94 | 1  | 1  |  1 | 1 |
|  Overall Accuracy |  0.92 |  0.92 | 0.9  | 0.94  |  0.91 | 0.92  | 0.95|